# 어노잉오렌지 전, 스노우 카메라 따라하기
스노우카메라 리뷰로 좀더 기본적인 동영상 코드를 알아보고 경험해 본다.

In [1]:
import cv2, dlib, sys
import numpy as np

## 1. 동영상에 bnd box 올려보기
dlib 라이브러리를 이용하여 얼굴 탐지

In [4]:
# resize (이미지 크기를 줄임)
scaler = 0.3

# initialize face detector and shape predictor
detector = dlib.get_frontal_face_detector()  # 얼굴 디텍터 모듈 초기화
# shape_pred.dat은 머신러닝으로 학습된 모델 
predictor = dlib.shape_predictor('../shape_predictor_68_face_landmarks.dat')  # 얼굴 특징점 모듈 초기화

detector, predictor   # objects

(<_dlib_pybind11.fhog_object_detector at 0x10d41c030>,
 <_dlib_pybind11.shape_predictor at 0x10d52e7b0>)

In [18]:
# load video(동영상 파일 로드)
# VideoCapture(0) - 파일 이름대신 0을 넣으면 웹캠
cap = cv2.VideoCapture('girl.mp4')
while True:
    # read frame buffer from video
    ret, img = cap.read()   # (boolean, np)
#     print(img.shape)   # (1080, 1920, 3) (h!, w, c) 
    if not ret:
        break
    
    # resize frame
    # resize(img, dsize) 크기 조절(꼭 정수형으로 값을 받아야함)
    img = cv2.resize(img, (int(img.shape[1] * scaler), int(img.shape[0] * scaler)))   # w, h 순서로 넣어야 함
#     print(img.shape)    # (324, 576, 3) (h, w, c)
    ori = img.copy()
    
    # 얼굴 찾기
    faces = detector(img)   # rectangles라는 dlib 객체, 인덱싱 가능
#     print(type(faces), faces)
    face = faces[0]    # 여러얼굴이 나오니까 첫번째 얼굴만
#     print(face)     # [(237, 83) (323, 170)], 좌상단 우하단
    
    # draw
    # rectangle(img, 좌상단좌표, 우하단 좌표)
#     print(type(face.left()), face.left())    # <class 'int'> 237
    img = cv2.rectangle(img, 
                        pt1=(face.left(), face.top()),   # 다음 메소드를 통해 int로 반환가능
                        pt2=(face.right(), face.bottom()),
                        color=(255, 255, 255),           # 흰색
                        thickness=2,
                        lineType=cv2.LINE_AA
                       )

    # visualize
    cv2.imshow('img', img)    # 윈도우 이름과 이미지 파일
    cv2.waitKey(1)

KeyboardInterrupt: 

## 2. bnd box +  facial landmark 올려보기
dlib 라이브러리를 이용하되, 얼굴 탐지는 라이브러리를 이용, 랜드마크 탐지는 사전 학습된 모델로 predictor로 만들어 사용

In [25]:
# resize (이미지 크기를 줄임)
scaler = 0.3

# initialize face detector and shape predictor
detector = dlib.get_frontal_face_detector()  # 얼굴 디텍터 모듈 초기화
# shape_pred.dat은 머신러닝으로 학습된 모델 
predictor = dlib.shape_predictor('../shape_predictor_68_face_landmarks.dat')  # 얼굴 특징점 모듈 초기화

# load video(동영상 파일 로드)
# VideoCapture(0) - 파일 이름대신 0을 넣으면 웹캠
cap = cv2.VideoCapture('girl.mp4')
while True:
    # read frame buffer from video
    ret, img = cap.read()
    if not ret:
        break
    
    # resize frame
    # resize(img, dsize) 크기 조절(꼭 정수형으로 값을 받아야함)
    img = cv2.resize(img, (int(img.shape[1] * scaler), int(img.shape[0] * scaler)))
    ori = img.copy()
    
    # 얼굴 찾기
    faces = detector(img)
    face = faces[0]  # 여러얼굴이 나오니까 첫번째 얼굴만
    
    # 얼굴 특징점(facial landmark) 찾기
    dlib_shape = predictor(img, face)   # dlib 객체
    shape_2d = np.array([[p.x, p.y] for p in dlib_shape.parts()])  # dlib 객체를 68개 2차원 np 좌표로 변환
#     print(type(shape_2d), shape_2d.shape, shape_2d)    #  <class 'numpy.ndarray'> (68, 2) [[246 100] ...[]]
    
    # 얻은 특징점으로 얼굴 size구하기(좌표 구하기)
    # 좌상단 우하단
    top_left = np.min(shape_2d, axis=0)    # 0차원끼리 비교
    bottom_right = np.max(shape_2d, axis=0)
#     print(top_left, bottom_right)    # [246  84] [328 172]
    
    # compute face center
    # 랜드마크로 얼굴의 중심(좌표 구하기)
    # 모든 특징점의 평균을 구하여 얼굴 중심을 구함, 정수형으로 변환필요
    center_x, center_y = np.mean(shape_2d, axis=0).astype(np.int)
    
    
    # draw
    # rectangle(img, 좌상단좌표, 우하단 좌표)
    img = cv2.rectangle(img, 
                        pt1=(face.left(), face.top()),
                        pt2=(face.right(), face.bottom()),
                        color=(255, 255, 255), # 흰색
                        thickness=2,
                        lineType=cv2.LINE_AA
                       )
    # 랜드마크를 원으로 표시
    for s in shape_2d:
        cv2.circle(img, center=tuple(s), radius=1, color=(255, 255, 255), thickness=2, lineType=cv2.LINE_AA)

    # 랜드마크의 좌상단 우하단 크기 그리기
    cv2.circle(img, center=tuple(top_left), radius=1, color=(255, 0, 0), thickness=2, lineType=cv2.LINE_AA)
    cv2.circle(img, center=tuple(bottom_right), radius=1, color=(255, 0, 0), thickness=2, lineType=cv2.LINE_AA)
    cv2.circle(img, center=(center_x, center_y), radius=1, color=(0, 0, 255), thickness=2, lineType=cv2.LINE_AA)


    # visualize
    cv2.imshow('img', img)    # 윈도우 이름과 이미지 파일
    cv2.waitKey(1)

/var/folders/_f/txsmx7r936jfbdwthf8dq_jm0000gn/T/ipykernel_91805/4091018938.py:41: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  center_x, center_y = np.mean(shape_2d, axis=0).astype(np.int)


## 3. 라이언얼굴로 바꾸기
overlay_transparent란 사용자정의함수 만들어 동영상에 정적 img overlay

In [28]:
# overlay function
def overlay_transparent(background_img, img_to_overlay_t, x, y, overlay_size=None):
  bg_img = background_img.copy()
  # convert 3 channels to 4 channels
  if bg_img.shape[2] == 3:
    bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGR2BGRA)

  if overlay_size is not None:
    img_to_overlay_t = cv2.resize(img_to_overlay_t.copy(), overlay_size)

  b, g, r, a = cv2.split(img_to_overlay_t)

  mask = cv2.medianBlur(a, 5)

  h, w, _ = img_to_overlay_t.shape
  roi = bg_img[int(y-h/2):int(y+h/2), int(x-w/2):int(x+w/2)]

  img1_bg = cv2.bitwise_and(roi.copy(), roi.copy(), mask=cv2.bitwise_not(mask))
  img2_fg = cv2.bitwise_and(img_to_overlay_t, img_to_overlay_t, mask=mask)

  bg_img[int(y-h/2):int(y+h/2), int(x-w/2):int(x+w/2)] = cv2.add(img1_bg, img2_fg)

  # convert 4 channels to 4 channels
  bg_img = cv2.cvtColor(bg_img, cv2.COLOR_BGRA2BGR)

  return bg_img

In [31]:
# resize (이미지 크기를 줄임)
scaler = 0.3

# initialize face detector and shape predictor
detector = dlib.get_frontal_face_detector()  # 얼굴 디텍터 모듈 초기화
# shape_pred.dat은 머신러닝으로 학습된 모델 
predictor = dlib.shape_predictor('../shape_predictor_68_face_landmarks.dat')  # 얼굴 특징점 모듈 초기화

# load video(동영상 파일 로드)
# VideoCapture(0) - 파일 이름대신 0을 넣으면 웹캠
cap = cv2.VideoCapture('girl.mp4')
# load overlay image(라이언 이미지)
overlay = cv2.imread('ryan_transparent.png', cv2.IMREAD_UNCHANGED) # 알파채널까지 함께 읽어야 함.(투명)
# print(type(overlay), overlay.shape)    # <class 'numpy.ndarray'> (130, 129, 4) (h, w, c) 알파채널 포함 4


while True:
    # read frame buffer from video
    ret, img = cap.read()
    if not ret:
        break
    
    # resize frame
    # resize(img, dsize) 크기 조절(꼭 정수형으로 값을 받아야함)
    img = cv2.resize(img, (int(img.shape[1] * scaler), int(img.shape[0] * scaler)))
    ori = img.copy()
    
    # 얼굴 찾기
    faces = detector(img)
    face = faces[0]  # 여러얼굴이 나오니까 첫번째 얼굴만
    
    # 얼굴 특징점(facial landmark) 찾기
    dlib_shape = predictor(img, face)
    shape_2d = np.array([[p.x, p.y] for p in dlib_shape.parts()])  # dlib 객체를 np로 변환
    
    # 얻은 특징점으로 얼굴 size구하기(좌표 구하기)
    # 좌상단 우하단
    top_left = np.min(shape_2d, axis=0)
    bottom_right = np.max(shape_2d, axis=0)
#     print(top_left, bottom_right)      # [246  84] [328 172]
    
    # compute face size
    # 우하단에서 좌상단뺀 x, y길이의 가장 긴 값
    face_size = max(bottom_right - top_left)     # 82 88
    print(face_size)
    # 라이언 얼굴 사이즈를 키우기 위해 1.8배 
    face_size = int(np.mean(face_size) * 1.8)
#     print(face_size)
    
    # compute face center
    # 랜드마크로 얼굴의 중심(좌표 구하기)
    # 모든 특징점의 평균을 구하여 얼굴 중심을 구함, 정수형으로 변환필요
    center_x, center_y = np.mean(shape_2d, axis=0).astype(np.int)
    
    # draw overlay on face
    # 라이언이미지를 센터x, y를 중심으로 넣고 overlay사이즈(얼굴의 가로세로중 가장큰 크기)만큼 resize해서 원본이미지에 넣어줌
    result = overlay_transparent(ori, overlay, 
                                 center_x + 8, center_y - 25, 
                                 overlay_size=(face_size, face_size))
    
    
    # draw
    # rectangle(img, 좌상단좌표, 우하단 좌표)
    img = cv2.rectangle(img, 
                        pt1=(face.left(), face.top()),
                        pt2=(face.right(), face.bottom()),
                        color=(255, 255, 255), # 흰색
                        thickness=2,
                        lineType=cv2.LINE_AA
                       )
    # 랜드마크를 원으로 표시
    for s in shape_2d:
        cv2.circle(img, center=tuple(s), radius=1, color=(255, 255, 255), thickness=2, lineType=cv2.LINE_AA)

    # 랜드마크의 좌상단 우하단 크기 그리기
    cv2.circle(img, center=tuple(top_left), radius=1, color=(255, 0, 0), thickness=2, lineType=cv2.LINE_AA)
    cv2.circle(img, center=tuple(bottom_right), radius=1, color=(255, 0, 0), thickness=2, lineType=cv2.LINE_AA)
    cv2.circle(img, center=(center_x, center_y), radius=1, color=(0, 0, 255), thickness=2, lineType=cv2.LINE_AA)


    # visualize
    cv2.imshow('img', img)    # 윈도우 이름과 이미지 파일
    cv2.imshow('result', result)  # 라이언 이미지 오버랩 이미지
    cv2.waitKey(1)

88
88
88
88
88
88


/var/folders/_f/txsmx7r936jfbdwthf8dq_jm0000gn/T/ipykernel_91805/982654803.py:53: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  center_x, center_y = np.mean(shape_2d, axis=0).astype(np.int)


88
88
88
88
88
88
87
87
87
87
87
87
87
87
87
87
87
87
88
87
87
87
87
87
87
87
87
87
87
87
88
87
87
87
88
87
88
88
87
87
88
88
87
87
87
88
88
88
87
87
87
88
87
87
87
87
87
87
87
87
87
87
88
88
87
87
87
87
87
87
88
87
87
87
87
87
87
87
87
86
87
87
87
87
87
87
87
87
88
88
87
87
87
87
87
87
87
87
86
87
87
87
87
88
87
87
88
88
88
88
87
87
88
88
88
88
87
88
87
87
87
87
87
86
87
87
86
87
87
87
88
87
87
87
87
87
88
88
87
88
87
88
88
88
88
87
88
87
88
88
88
87
88
88
88
87
88
88
88
87
88
88
87
87
87
87
88
88
87
87
88
88
87
87
88
87
87
87
87
87
89
88
88
88
87
88
88
88
88
88
88
88
88
88
88
88
87
88
88
88
88
88
88
88
88
88
87
87
88
88
88
89
88
88
88
87
87
88
87
88
87
87
87
88
88
88
87
87
88
88
88
88
87
87
87
87
88
88
88
88
87
87
88
88
88
88
87
87
87
87
87
88
88
88
88
88
88
88
87
87
88
88
88
88
87
87
88
88
88
88
87
87
87
87
88
88
87
87
88
88
88
88
88
88
88
88
88
88
89
88
88
88
88
88
88
88
88
88
87
87
88
88
89
87
88
88
88
88
88
88
88
88
88
88
87
87
87
87
88
88
87
87
88
88
88
88
88
88
88
88
88
88
88
8